In [2]:
import librosa
import numpy as np
from IPython.display import Audio

In [3]:
y, srs=librosa.load("Desktop/hello.ogg")  

In [4]:
#playing the audio
Audio(data=y, rate=srs)

In [5]:
def quantisation(steps, signal):
    maxm = max(signal)                
    minm = min(signal)                  
    step_size = float(maxm - minm) / (steps) 
    quantisation_intervals = np.zeros((steps, 2))
    quantised_signal = np.array([])
    lower = minm
    for i in range(steps):
        upper = lower + step_size
        quantisation_intervals[i] = [lower, upper]
        lower += step_size
    for i in range(len(signal)):
        for j in range(steps):
            if (signal[i] >= quantisation_intervals[j, 0] and signal[i] < quantisation_intervals[j, 1]):
                quantised_signal = np.append(quantised_signal, (quantisation_intervals[j, 0] + quantisation_intervals[j, 1])/2)
            elif (signal[i] == quantisation_intervals[steps-1, 1]):
                quantised_signal = np.append(quantised_signal, (quantisation_intervals[steps-1, 0] + quantisation_intervals[steps-1, 1])/2)
                break
    return quantised_signal

In [6]:
steps=64
quantised_audio_data = quantisation(steps, y)

In [9]:
def encoding_dictionary(signal,steps):
    maxm = max(signal)                  
    minm = min(signal)                 
    step_size = float(maxm - minm) / (steps) 
    quantisation_intervals = np.zeros((steps, 2))
    code = {}
    lower = minm
    for i in range(steps):
        upper = lower + step_size
        quantisation_intervals[i] = [lower, upper]
        lower += step_size
    for i in range(steps):
        c = bin(i)[2:]
        while(len(c)!=np.log2(steps)):
            c = '0'+c
        code[(quantisation_intervals[i, 0] + quantisation_intervals[i, 1])/2] = c
    return code

In [15]:
codes = encoding_dictionary(y,steps)

In [16]:
def coded_signal(quantised_signal,code):
    coded_sequence = []
    for value in quantised_signal:
        for i in code.keys():
            if(value == i):
                coded_sequence.append(code[i])
                break
    return coded_sequence

In [17]:
pcm_signal_list = coded_signal(quantised_audio_data,codes)

In [18]:
pcm_signal_string = ''.join(l for l in pcm_signal_list)

In [19]:
n=7
k=4
split_pcm_coded = []
for i in range(0,len(pcm_signal_string),k):
    split_pcm_coded.append(pcm_signal_string[i:i+k])

while len(split_pcm_coded[-1])%k != 0:
    split_pcm_coded[-1] = '0' + split_pcm_coded[-1]

In [20]:
P = [[0,1,1],[1,0,1],[1,1,0],[1,1,1]]
G = np.eye(k).tolist()
for i in range(k):
    G[i] = G[i] + P[i]
    G[i] = list(map(int, G[i]))

In [21]:
P_T = np.array(P).T
P_T = P_T.tolist()

H = np.eye(n-k).tolist()
for i in range(np.shape(P_T)[0]):
    H[i] = P_T[i] + H[i]
    H[i] = list(map(int, H[i]))

In [22]:
def hamming_encoding(u,G):
    u = list(map(int,u))
    ham_encoded = list(map(int,np.zeros(n)))
    for i in range(k):
        for j in range(n):
            ham_encoded[j] = ham_encoded[j]^(u[i] & G[i][j])
    ham_encoded = ''.join(str(l) for l in ham_encoded)
    return ham_encoded

In [23]:
hamming_coded_sequence = ''
hamming_coded_list = []
for u in split_pcm_coded:
    v = hamming_encoding(u,G)
    hamming_coded_sequence += v
    hamming_coded_list.append(v)

In [24]:
m=4
Es = 1
theoretical_points = {}
for i in range(m):
    mpsk_symbol = np.sqrt(Es)*np.cos(2*np.pi*(i-1)/m) + 1j*np.sqrt(Es)*np.sin(2*np.pi*(i-1)/m)
    key = bin(i)[2:]
    while(len(key) != int(np.log2(m))):
        key = '0' + key
    theoretical_points[key] = mpsk_symbol.round(4)
bit_signal = []
for i in range(0,len(hamming_coded_sequence),int(np.log2(m))):
    bit_signal.append(hamming_coded_sequence[i:i+int(np.log2(m))])
while len(bit_signal[-1])%m != 0:
    bit_signal[-1] = '0' + bit_signal[-1]
bit_signal_str = ""
for i in range(len(bit_signal)):
    bit_signal_str += str(bit_signal[i])

In [25]:
def symbol(transmitted_str,m,theoretical_points):
    bits = int(np.log2(m))
    symbol_array = []
    for i in range(0, int(len(transmitted_str)),bits):
        val = transmitted_str[i:i+bits]
        symbol_array.append(theoretical_points[val])
    return symbol_array

In [26]:
symbol_array = symbol(bit_signal_str,m,theoretical_points)

In [28]:
def noise(db,symbol_array):
    std_dev = np.sqrt(1/(10**(db/10)))
    awgn = np.random.normal(0,std_dev/np.sqrt(2),len(symbol_array)) + 1j*np.random.normal(0,std_dev/np.sqrt(2),len(symbol_array))
    return awgn

In [29]:
received_symbol_array = symbol_array + noise(12,symbol_array)

In [30]:
def ML_decoder(received_symbol_array, m, theoretical_points, Es=1):
    decoded_symbol_array = []
    for i in range(len(received_symbol_array)):
        dic = {}
        for j in range(m):
            key = bin(j)[2:]
            while len(key)%int(np.log2(m)) != 0:
                key = '0' + key
        
            d = np.sqrt((np.real(theoretical_points[key]) - np.real(received_symbol_array[i]))**2 + (np.imag(theoretical_points[key]) - np.imag(received_symbol_array[i]))**2)
            dic[key] = d
        min_dist = min(list(dic.values()))
        for k in range(m):
            key = bin(k)[2:]
            while len(key)%int(np.log2(m)) != 0:
                key = '0' + key
            if min_dist == dic[key]:
                decoded_symbol_array.append(theoretical_points[key])
    return decoded_symbol_array

In [31]:
demodulated_symbol_array = ML_decoder(received_symbol_array, m, theoretical_points, Es=1)

In [32]:
def demodulated_binary(demodulated_symbol_array,m,theoretical_points):
    decoded_bit_string = ""
    bits = int(np.log2(m))
    for i in range(len(demodulated_symbol_array)):
        for j in range(m):
            key = bin(j)[2:]
            while len(key)%bits != 0:
                key = '0' + key
            if demodulated_symbol_array[i] == theoretical_points[key]:
                decoded_bit_string += key
    return decoded_bit_string

In [33]:
demodulated_bit_string = demodulated_binary(demodulated_symbol_array,m,theoretical_points)

In [34]:
demodulated_bit_list = []
for i in range(0,len(demodulated_bit_string),n):
    demodulated_bit_list.append(demodulated_bit_string[i:i+n])

while len(demodulated_bit_list[-1]) != n:
    demodulated_bit_list[-1] = '0' + demodulated_bit_list[-1]

In [35]:
def ham_decoding_dict(n,k,G):
    ham_decoding = {}
    for l in range(2**k):
        l = bin(l)[2:]
        while len(l) != k:
            l = '0'+l
        l = list(map(int,l))
        ham_encoding = list(map(int,np.zeros(n)))
        for i in range(k):
            for j in range(n):
                ham_encoding[j] = ham_encoding[j]^(l[i] & G[i][j])
        key = ''.join(str(b) for b in ham_encoding)
        value = ''.join(str(b) for b in l)
        ham_decoding[key] = value
    return ham_decoding

In [36]:
hamming_decodings_dictionary = ham_decoding_dict(n,k,G)

In [37]:
def syndrome_dict(n,k,H):
    H_T = np.array(H).T
    error_matrix = np.eye(n).tolist()
    for i in range(len(error_matrix)):
        error_matrix[i] = list(map(int,error_matrix[i]))
    syndrome = {}
    for g in range(n):
        syn_value = list(map(int,np.zeros((n-k))))
        for j in range(n-k):
            for i in range(n):
                syn_value[j] = syn_value[j]^(error_matrix[g][i] & H_T[i][j])
                key = ''.join(str(l) for l in error_matrix[g])
                value = ''.join(str(l) for l in syn_value)
                syndrome[key] = value
    key = '0'*n
    value = '0'*(n-k)
    syndrome[key] = value
    return syndrome

In [38]:
syndrome_dictionary = syndrome_dict(n,k,H)

In [39]:
def hamming_decoding(ml_decoded_string, H, syndrome_dictionary, hamming_decodings_dictionary):
    ml_decoded_list = list(map(int,ml_decoded_string))
    H_T = np.array(H).T
    syndrome_decoded_list = list(map(int,np.zeros(n)))
    syn = list(map(int,np.zeros(n-k)))
    for j in range(n-k):
        for i in range(n):
            syn[j] = syn[j]^(ml_decoded_list[i] & H_T[i][j]) 
    syn = ''.join(str(l) for l in syn)
    for key in list(syndrome_dictionary.keys()):
        if syndrome_dictionary[key] == syn:
            error = key   
    error = list(map(int,error)) 
    for i in range(len(ml_decoded_list)):
        syndrome_decoded_list[i] = ml_decoded_list[i]^error[i]
    syndrome_decoded_string = ''.join(str(i) for i in syndrome_decoded_list)
    hamming_decoded_string = hamming_decodings_dictionary[syndrome_decoded_string]
    return hamming_decoded_string

In [40]:
hamming_decoded_string = ''
for i in demodulated_bit_list:
    hamming_decoded_string += hamming_decoding(i,H,syndrome_dictionary,hamming_decodings_dictionary)

In [42]:
codes = encoding_dictionary(y,steps)

In [43]:
received_pcm_list = []
bits = int(np.log2(steps))
for i in range(0, len(hamming_decoded_string), bits):
    received_pcm_list.append(hamming_decoded_string[i:i+bits])
while len(received_pcm_list[-1]) != bits:
    received_pcm_list[-1] = '0' + received_pcm_list[-1]

In [44]:
decoded_pcm_list = []
for i in range(len(received_pcm_list)):
    for j in list(codes.keys()):
        if codes[j] == received_pcm_list[i]:
            decoded_pcm_list.append(j)

In [46]:
Audio(data=decoded_pcm_list, rate=srs)